In [ ]:
# training:
#   root_dir: artifacts/training
#   trained_model_path: artifacts/training/model.h5

In [1]:
import os
%pwd

'd:\\projects\\MLOps\\Sign-Language-Recognition\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\projects\\MLOps\\Sign-Language-Recognition'

In [3]:
#Entity 

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [4]:
from Sign_Language.constants import *
from Sign_Language.utils.common import create_directories, read_yaml

In [5]:
# Configuration Manager

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:

        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])


        get_prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
        )

        return get_prepare_callbacks_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params.baseModelParams

        training_data = Path(self.config.data_ingestion.train_data_path)

        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            base_model_path = Path(prepare_base_model.base_model_path),
            training_data = training_data,
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            )
        
        return training_config

In [6]:
import time
import os
import tensorflow as tf
import pandas as pd
import numpy as np

In [7]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [8]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.base_model_path)
    
    def train_valid_generator(self):

        data = pd.read_csv(self.config.training_data)
        self.X = np.array(data.drop('label', axis = 1))
        self.y = np.array(data[['label']])
        
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        self.model.fit(self.X, self.y, batch_size=self.config.params_batch_size, 
                       epochs=self.config.params_epochs, validation_split=0.2, 
                       callbacks=callback_list)
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [9]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-02-11 00:34:59,537: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-11 00:34:59,543: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-11 00:34:59,547: INFO: common: created directory at: artifacts]
[2024-02-11 00:34:59,549: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-02-11 00:34:59,549: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-02-11 00:34:59,549: INFO: common: created directory at: artifacts\training]



Epoch 1/10


341/341 [==============================] - 14s 32ms/step - loss: 1.7265 - accuracy: 0.5179 - val_loss: 0.7637 - val_accuracy: 0.8333
Epoch 2/10
  1/341 [..............................] - ETA: 29s - loss: 0.8327 - accuracy: 0.7700

c:\Users\mp123\anaconda3\envs\aslenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


341/341 [==============================] - 10s 28ms/step - loss: 0.6431 - accuracy: 0.8230 - val_loss: 0.5164 - val_accuracy: 0.8810
Epoch 3/10
341/341 [==============================] - 9s 27ms/step - loss: 0.5066 - accuracy: 0.8615 - val_loss: 0.4228 - val_accuracy: 0.8969
Epoch 4/10
341/341 [==============================] - 12s 36ms/step - loss: 0.4352 - accuracy: 0.8767 - val_loss: 0.3492 - val_accuracy: 0.9159
Epoch 5/10
341/341 [==============================] - 11s 33ms/step - loss: 0.3892 - accuracy: 0.8903 - val_loss: 0.3200 - val_accuracy: 0.9204
Epoch 6/10
341/341 [==============================] - 10s 30ms/step - loss: 0.3541 - accuracy: 0.8976 - val_loss: 0.2968 - val_accuracy: 0.9220
Epoch 7/10
341/341 [==============================] - 14s 40ms/step - loss: 0.3279 - accuracy: 0.9055 - val_loss: 0.2795 - val_accuracy: 0.9226
Epoch 8/10
341/341 [==============================] - 12s 35ms/step - loss: 0.3060 - accuracy: 0.9088 - val_loss: 0.2592 - val_accuracy: 0.9293
Epoc